## Benchmark IN-trained classifiers on `ImageNet-Hard`

<a target="_blank" href="https://colab.research.google.com/github/taesiri/ZoomIsAllYouNeed/blob/main/src/ImageNet_Hard/Benchmark-ImageNet-Hard.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

* [Website](https://taesiri.github.io/ZoomIsAllYouNeed/)
* [Github](https://github.com/taesiri/ZoomIsAllYouNeed)

In [3]:
!pip install transformers datasets timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.6 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset

In [5]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(device=0))
print(torch.__version__)

1
Tesla T4
1.13.1+cu116


## Transforms

In [6]:
standard_transform = transforms.Compose(
  [transforms.Resize(256), 
   transforms.CenterCrop(224), 
   transforms.ToTensor(), 
   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

## Datasets


In [7]:
imagenet_hard_dataset = load_dataset('taesiri/imagenet-hard', split='validation')
imagenet_hard_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/23845 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/taesiri___parquet/taesiri--imagenet-hard-124be08d1e33678b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Dataset({
    features: ['image', 'label'],
    num_rows: 23845
})

In [8]:
def preprocess_batch(batch):
    batch['image'] = [standard_transform(image.convert('RGB')) for image in batch['image']]
    return batch

imagenet_hard_dataset.set_transform(preprocess_batch)

In [9]:
# Indices of classes that should be masked out (set to False) in ImageNet's 1000 classes
false_mask = [ 12,  13,  24, 333, 339, 340, 352, 354, 386, 400, 404, 430, 444, 466, 510, 527, 630, 668, 746, 779, 802, 890, 916, 919, 954, 981, 984, 985]
mask = np.ones(1000, dtype=bool)
mask[false_mask] = False

In [10]:
# Helpers
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to('cpu').numpy()

## Benchmark

In [11]:
def run_benchmark_masked(model, bs=16):
  model.cuda()
  model.eval()
  
  
  loader = DataLoader(imagenet_hard_dataset, batch_size=256, num_workers=2)

  correct_ones = 0
  with torch.inference_mode():
    with torch.no_grad():
      for i, (batch) in enumerate(tqdm(loader)):
        images, target = batch['image'], batch['label']
        images = images.cuda()
        target = target.cuda()
        
        model_output = model(images)[:,mask]
        pred = model_output.data.max(1)[1]
        correct_ones += pred.eq(target.data).sum().item()
  return 100*correct_ones/len(imagenet_hard_dataset)

In [12]:
model_names = ['resnet50', 'resnet18', 'alexnet', 'vgg19', 'vit_b_32']

In [13]:
accuracy = {}

for name in model_names:
  model = models.__dict__[name](pretrained=True)
  accuracy[name] = run_benchmark_masked(model)
  print(f'{name} accuracy: {accuracy[name]}')

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

100%|██████████| 94/94 [05:46<00:00,  3.69s/it]
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


resnet50 accuracy: 11.52023485007339


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


...

vgg19 accuracy: 9.00398406374502


Downloading: "https://download.pytorch.org/models/vit_b_32-d86f8d99.pth" to /root/.cache/torch/hub/checkpoints/vit_b_32-d86f8d99.pth


  0%|          | 0.00/337M [00:00<?, ?B/s]

100%|██████████| 94/94 [05:28<00:00,  3.49s/it]

vit_b_32 accuracy: 16.682742713357097


In [14]:
pd.DataFrame(accuracy, index=['accuracy']).T.round(2)

,accuracy
resnet50,11.52
resnet18,8.65
alexnet,5.63
vgg19,9.00
vit_b_32,16.68
